In [112]:
# requests 라이브러리 불러 오기
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [179]:
import time

In [183]:
def parse_date(txt):
    date_begin = re.compile('categories: \[')
    date_begin = date_begin.search(txt).span()[1]
    date_end = re.compile('\]},yAxis:')
    date_end = date_end.search(txt).span()[0]
    
    return txt[date_begin:date_end].replace('"','').split(',')
    
    
def parse_data(txt):
    data_begin = re.compile('data: \[')
    data_begin = data_begin.search(txt).span()[1]
    data_end = re.compile('\]}\],responsive')
    data_end = data_end.search(txt).span()[0]
    
    return txt[data_begin:data_end].split(',')
    

In [215]:
def txt_to_html(txt):
    case_compile = re.compile('Total Coronavirus Cases')
    death_compile = re.compile('Total Coronavirus Deaths')
    
    for script in txt.find_all(attrs={'type':"text/javascript"}):
        try:
            if bool(case_compile.search(script.contents[0])):
                total_case = script.contents[0]
            elif bool(death_compile.search(script.contents[0])):
                total_death = script.contents[0]
        except:
            pass
        
#    total_case = txt.find_all(attrs={'type':"text/javascript"})[6].contents[0]
    total_case  = total_case.replace('\n','').replace('\'', '"').replace('    ','')
 #   total_death = txt.find_all(attrs={'type':"text/javascript"})[8].contents[0]
    total_death  = total_death.replace('\n','').replace('\'', '"').replace('    ','')
    
    case_date = parse_date(total_case)
    case_data = parse_data(total_case)
    death_date = parse_date(total_death)
    death_data = parse_data(total_death)
   
    return {"case_date":case_date, 'case_data':case_data,
            'death_date': death_date, 'death_data':death_data}

In [227]:
def crawl_country(cnts):
    address = 'https://www.worldometers.info/coronavirus/country/'
    outcome = pd.DataFrame()
    
    for cnt in cnts:
        web_add = address+'{}'.format(cnt)
        html = requests.get(web_add)
        soup = bs(html.text, 'html.parser')
        html.close()

        tmp = txt_to_html(soup)
        tmp = pd.DataFrame(tmp)
        tmp.loc[:,'country'] = cnt
    
        outcome = pd.concat([outcome, tmp])
    
    outcome.case_data = outcome.case_data.astype(int)
    outcome.death_data = outcome.death_data.astype(int)
    return outcome

In [228]:
cnt_list = ['UK', 'France', 'Germany', 'Spain', 'Australia']

In [229]:
outcome = crawl_country(cnt_list)

In [230]:
outcome.to_excel('tmp.xlsx', index=False)